# Lab 09: Word embeddings (2/2)

<br>

<img width=300 align="left" margin="20" src="number-matrix.jpg">
<img width=300 margin="20" src="wordgeom.png">

<br>

In this lab we will continue to work with word embeddings&mdash;machine learning algorithms that represent categorical data like words as vectors in a high dimensional space. 

Embeddings are constructed using cooccurrence statistics, and can be applied whenever you have "objects" that appear together&mdash;songs in playlists, words in documents, ingredients in recipes... Embeddings can reveal surprising semantic relations encoded in linear relationships; but they are "data hungry" and require large corpora of text or other coocurrence data to construct useful representations.

We will first explore how to visualize the embedding vectors. Perhaps confusingly, this is done by embedding the embedding vectors themselves into two dimensions. The most 
popular method for doing this is called t-SNE ("TEE snee"). [Here](https://lvdmaaten.github.io/tsne/) is an overview of t-SNE by its creator, and [here](http://cs.stanford.edu/people/karpathy/tsnejs/) is an introduction that includes some (potentially) interesting examples. We will then explore ways in which embeddings may represent certain "cultural biases," which has been a topic of recent interest in the machine learning and AI communities.

Let's begin by loading in the 100-dimensional GloVe embeddings we worked with last time. Recall that these are trained using word cooccurrence counts from about 6 billion words of text from Wikipedia pages.

In [ ]:
import gensim
import gensim.downloader as gdl
from gensim.models import KeyedVectors
glove = gdl.load("glove-wiki-gigaword-100")

Next we'll import the usual modules, and in addition a module that implements t-SNE.

In [ ]:
from datascience import *
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

%matplotlib inline

We'll now define two functions. The first takes a pre-trained embedding model, together with a vocabulary of words (as strings), which is a subset of the words in the embedding vocabulary. It then runs t-SNE on that vocabulary, displaying the results as a point cloud. The second function displays the location of particular words, so that we can get a sense of the geometry of the embeddings, and what's close to what.

In [ ]:

def generate_tsne_for_voc(model, voc):
    word_list = [w for w in model.vocab]
    w = word_list[0]
    dim = len(model[w])
    print("embedding dimension: %d" % dim)
    print("computing t-SNE vectors over %d words..." % len(voc))
    
    # collect all the vectors in a list
    arr = np.empty((0,dim), dtype='f')
    for w in voc:
        wrd_vector = model[w]
        arr = np.append(arr, np.array([wrd_vector]), axis=0)
        
    # find tsne coords for 2 dimensions
    tsne = TSNE(n_components=2, random_state=0)
    np.set_printoptions(suppress=True)
    Y = tsne.fit_transform(arr)

    # display scatter plot of the 2-d vectors
    plt.figure(figsize=(15,15))    
    plt.scatter(Y[:,0], Y[:,1], s=2)
    plt.show()
    return (Y,voc)


# this function displays the "word cloud" of all points from the t-SNE vectors,
# and then labels the words in the array input_words

def display_tsne_words(Z, input_words, size1=2, size2=50, offset=5):
    Y = Z[0]
    voc = Z[1]
    x_coord = Y[:, 0]
    y_coord = Y[:, 1]
    # display scatter plot
    plt.figure(figsize=(15,15))
    plt.scatter(x_coord, y_coord, s=size1)
    for w in np.arange(len(input_words)):
        index = voc.index(input_words[w])
        plt.scatter(x_coord[index], y_coord[index],s=size2)
        plt.annotate(input_words[w], xy=(x_coord[index],y_coord[index]), \
                     xytext=(offset,offset), textcoords='offset points')
    plt.show()
    


The following bit of code computes word frequencies. We read in the 'text8' dataset from last lab, which you'll recall is a few million words of Wikipedia text. We then build a vocabulary over this data using the `Counter` class.

In [ ]:
from collections import Counter
import time
import nltk

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
logging.root.level = logging.CRITICAL 

s = time.time()
voc = Counter()
for rec in open ('../lab08/text8', 'r'):
    rec = rec.strip()
    voc.update(rec.split())
print(time.time()-s)

Now we grab the most common words, excluding the top (say) 100.

In [ ]:
n = 5000
words = voc.most_common(n+100)[100:]
vocab = [word[0] for word in words]
len(vocab)

Finally, we run t-SNE on the GloVe embeddings, but restricting to this subset of the model vocabulary. This is only for speed and memory considerations.

In [ ]:
%%time
filtered_voc = [w for w in vocab if w in glove.vocab]
len(filtered_voc)
plt.figure(figsize=(15,15))
Z = generate_tsne_for_voc(glove, filtered_voc)

Now, we display some words by labeling points in the t-SNE embeddings.

In [ ]:
display_tsne_words(Z, input_words=['conservative', 'liberal'])

### Your turn!

Using the functions defined above, generate several plots show locations of points you choose. Generate a plot, and then describe in a markdown cell why this either makes sense or is puzzling to you. For example, you might find that "apple" is not near "fruit" but is rather close to "ibm." This should be easy to explain. Find others and comment on what you see.

To start, first generate t-SNE vectors for a larger subset of the 400,000 model vocabulary. You might use something like `n=10000`.